In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import os
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import pandas as pd
od.download("https://www.kaggle.com/datasets/vbookshelf/pneumothorax-chest-xray-images-and-masks/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gigasingh23
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/vbookshelf/pneumothorax-chest-xray-images-and-masks


100%|██████████| 4.50G/4.50G [02:01<00:00, 39.6MB/s]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
import multiprocessing
multiprocessing.cpu_count()

2

In [ ]:
class PneumoniaDataset(data.Dataset):
    def __init__(self, root_dir, csv_file, transform=None):
        self.root_dir = root_dir
        self.csv_file = csv_file
        self.transform = transform
        self.data_info = pd.read_csv(csv_file)

    def __getitem__(self, index):
        image_id = self.data_info.iloc[index, 0]
        label = self.data_info.iloc[index, 2]
        image_path = os.path.join(self.root_dir, 'png_images', f'{image_id}')

        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.data_info)

In [ ]:
# Example usage:
root_dir = './pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/'
train_csv_file = './pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_train_images.csv'
test_csv_file = './pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_test_images.csv'
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

train_dataset = PneumoniaDataset(root_dir, train_csv_file, transform)
test_dataset = PneumoniaDataset(root_dir, test_csv_file, transform)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [int(len(train_dataset)*0.8), len(train_dataset)-int(len(train_dataset)*0.8)])

In [ ]:
# Get the labels for the train and test datasets
# train_labels = []
# test_labels = []
# val_labels = []

# for i in range(len(train_dataset)):
#     train_label = train_dataset[i][1]
#     train_labels.append(train_label)
#     print(f"Train dataset iteration: i = {i}")

# for i in range(len(test_dataset)):
#     test_label = test_dataset[i][1]
#     test_labels.append(test_label)
#     print(f"Test dataset iteration: i = {i}")

# for i in range(len(val_dataset)):
#     val_label = val_dataset[i][1]
#     val_labels.append(val_label)
#     print(f"Validation dataset iteration: i = {i}")

train_data = pd.read_csv('./pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_train_images.csv')
test_data = pd.read_csv('./pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/stage_1_test_images.csv')
v
train_labels = train_data

,new_filename,ImageId,has_pneumo
0,0_train_0_.png,1.2.276.0.7230010.3.1.4.8323329.5597.151787518...,0
1,1_train_0_.png,1.2.276.0.7230010.3.1.4.8323329.12515.15178752...,0
2,2_train_1_.png,1.2.276.0.7230010.3.1.4.8323329.4904.151787518...,1
3,3_train_1_.png,1.2.276.0.7230010.3.1.4.8323329.32579.15178751...,1
4,4_train_1_.png,1.2.276.0.7230010.3.1.4.8323329.1314.151787516...,1
...,...,...,...
10670,10670_train_1_.png,1.2.276.0.7230010.3.1.4.8323329.32459.15178751...,1
10671,10671_train_1_.png,1.2.276.0.7230010.3.1.4.8323329.4461.151787518...,1
10672,10672_train_0_.png,1.2.276.0.7230010.3.1.4.8323329.32730.15178751...,0
10673,10673_train_0_.png,1.2.276.0.7230010.3.1.4.8323329.13252.15178752...,0


In [ ]:
# Perform stratified split on the train_dataset
train_dataset, _ = train_test_split(
    train_dataset,
    test_size=0.6,
    stratify=train_labels,
    random_state=42
)

# Perform stratified split on the test_dataset
test_dataset, _ = train_test_split(
    test_dataset,
    test_size=0.6,
    stratify=test_labels,
    random_state=42
)

# Perform stratified split on the val_dataset
val_dataset, _ = train_test_split(
    val_dataset,
    test_size=0.6,
    stratify=val_labels,
    random_state=42
)

In [ ]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

3416
854
548


In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=multiprocessing.cpu_count())
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=multiprocessing.cpu_count())
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=multiprocessing.cpu_count())

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = torchvision.models.densenet121(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = True
        self.fc = nn.Linear(self.model.classifier.in_features, 1)

        self.model.classifier = self.fc

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
from torchvision.models import densenet121, DenseNet


In [ ]:
model = MyModel().to(device)

# Defining loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
print(model)

MyModel(
  (model): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, 

In [ ]:
  # Training loop
for epoch in range(1):
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    all_train_preds = []
    all_train_labels = []
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        all_train_preds.extend(predicted.cpu().numpy())
        all_train_labels.extend(labels.cpu().numpy())
    torch.save(model.state_dict(), f'model_weights_epoch_{epoch+1}.pth')

    with torch.no_grad():
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        all_val_preds = []
        all_val_labels = []
        for i, data in enumerate(val_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
            all_val_preds.extend(predicted.cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())
        print('Epoch %d, Loss: %.3f, Train Acc: %.3f, Val Acc: %.3f, Val Loss: %.3f' %
              (epoch+1, running_loss/(i+1), correct_train/total_train, correct_val/total_val, val_loss/(i+1)))

print('Finished Training')

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Calculate additional metrics after training is finished
train_tn, train_fp, train_fn, train_tp = confusion_matrix(all_train_labels, all_train_preds).ravel()
val_tn, val_fp, val_fn, val_tp = confusion_matrix(all_val_labels, all_val_preds).ravel()

train_pos_vs_gt = train_tp / (train_tp + train_fn)
train_specificity = train_tn / (train_tn + train_fp)
train_precision = train_tp / (train_tp + train_fp)

val_sensitivity = val_tp / (val_tp + val_fn)
val_specificity = val_tn / (val_tn + val_fp)
val_precision = val_tp / (val_tp + val_fp)

fpr, tpr, _ = roc_curve(all_val_labels, all_val_preds)
val_auroc = auc(fpr, tpr)

print(f'Train Sensitivity: {train_pos_vs_gt:.3f}, Train Specificity: {train_specificity:.3f}, Train Precision: {train_precision:.3f}')
print(f'Val Sensitivity: {val_sensitivity:.3f}, Val Specificity: {val_specificity:.3f}, Val Precision: {val_precision:.3f}')
print(f'Val AUROC: {val_auroc:.3f}')
print(f'Train Confusion Matrix: TN={train_tn}, FP={train_fp}, FN={train_fn}, TP={train_tp}')
print(f'Val Confusion Matrix: TN={val_tn}, FP={val_fp}, FN={val_fn}, TP={val_tp}')

In [ ]:
results = []
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)

        for i in range(len(inputs)):
            prob = probabilities[i][1].item()  # Probability of the positive class
            ground_truth = labels[i].item()
            label_str = 'pneumo' if predicted[i].item() == 1 else 'no_pneumo'
            results.append([prob, ground_truth, label_str])

        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy:.4f}")


In [ ]:
results_df = pd.DataFrame(results, columns = ['prob', 'ground_truth', 'label'])
results_df.to_csv('pneumothorax/subpneumo/pnemothorax_results.csv', index=False)

print('Testing and saving results completed.')